In [1]:
# source "C:/Users/leand/venvs/pns/Scripts/activate"
# python -m ipykernel install --user --name=pns --display-name "pns"

import os
import pandas as pd
import seaborn as sns

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os
from tqdm import tqdm

# Módulos de interesse

- Modulo A, B e D: Contêm informações sociodemográficas
- Modulo L: Crianças com menos de 2 anos
- Módulo R: Saúde da Mulher
- Modulo S: Pré-natal
- Modulo W: Antropometria


# Analise de Dados

In [2]:
modulo_identificacao = [
    # Módulo de Identificacao
    "V0001",  # Unidade da Federação
    "V0024", # Estrato
    "UPA_PNS", # UPA
    "V0006_PNS", # Número ordem domicilio PNS
    "C00301", # Número ordem do morador
    "V0015", # Tipo da Entrevista
    "V0022", # Quantidade de moradores
    "V0026",  # Tipo de situação censitária
    "V0031", # Tipo de área Capital, Metropolitana
    "V0025A", # Seleção do morador de 15 anos ou mais para responder o questionário individual
    "V0025B", # Seleção do morador de 15 anos ou mais para ter medida antropométrica aferida
    "V0029", # Peso amostral do morador sem calibração
    "V00291", # Peso amostral do morador com calibração
    "V00282", #		Projeção da população
    "V00283", #	Domínio de projeção para domicílio e moradores
    "V00293", # Domínio de projeção para morador selecionado
    "V00303", # Domínio de projeção para morador selecionado para antropometria
    ]
modulo_C = [
    # Módulo C - Caracteristicas gerais dos moradores
    "C001", # Quantidade de pessoas no domicilio
    "C006",  # Sexo
    "C00701",  # Dia de nascimento
    "C00702",  # Mês de nascimento
    "C00703",  # Ano de nascimento
    "C008",  # Idade do morador na data de referência
    "C009",  # Cor ou Raça
]
modulo_D = [
    # Módulo D - Cartacterisiticas de Educação dos Moradores
    "D00201",  # Frequenta escola ou creche
    "D00901",  # Qual foi o curso mais elevado que ___frequentou
]
modulo_E = [
    # Módulo E - Características de Trabalho
    "E01602",  # Qual era o rendimento bruto mensal ou retirada que ___ fazia normalmente nesse trabalho?(valor em dinheiro)
    "E017", # Quantas horas ___ trabalhava normalmente, por semana, nesse trabalho?
]
modulo_G = [
    # Módulo G - Deficiecias maiores de 2 anos
    "G033",  # ___ usa óculos ou outro aparelho de auxílio para lidar com problemas de visão? 
    "G047",  #  ___ tem dificuldade permanente de enxergar? 
    "G046",  # ___ tem dificuldade permanente de enxergar mesmo usando óculos, lentes de contato ou lupas?
    "G049",  # ____________ faz uso de aparelho auditivo
    "G051",  # ____________ faz uso de implante coclear?
    "G053",  # ____________ faz uso de sistema de frequência modulada individual (sistema FM)?
    "G059",  # ___ usa algum aparelho de auxílio para se locomover?
    "G060",  # ____________ faz uso de cadeira de rodas?
    "G062",  # ____________ faz uso de bengala, muletas ou andador?
    "G064",  # ____________ faz uso de prótese?
    "G066",  # ____________ faz uso de órtese?
    "G070",  # ___ tem dificuldade permanente de caminhar ou subir degraus, mesmo usando prótese, bengala ou outro aparelho de auxílio? (Para moradores com 5 anos ou mais de idade.) OU Comparado com crianças da mesma idade, ___ tem dificuldade permanente para caminhar, mesmo usando prótese, bengala ou aparelho de auxílio? (Para moradores com 2 a 4 anos de idade.)
    "G072",  # ___ usa algum aparelho de auxílio para realizar movimentos com os membros superiores?
    "G073",  # ____________ faz uso de prótese para os membros superiores?
    "G079",  #  ___tem dificuldade permanente para levantar uma garrafa com dois litros de água da cintura até a altura dos olhos, mesmo usando prótese ou aparelho de auxílio? (Somente para moradores com 5 anos ou mais de idade.)
    "G083",  # Por causa de alguma limitação nas funções mentais ou intelectuais, _________ tem dificuldade permanente para realizar atividades habituais, como se comunicar, realizar cuidados pessoais, trabalhar, ir à escola, brincar, etc.? (Para moradores com 5 anos ou mais de idade.) OU Por causa de alguma limitação nas funções mentais ou intelectuais, _________ tem dificuldade permanente para realizar atividades habituais, como frequentar a escola, brincar e etc.? (Para moradores com 2 a 4 anos de idade.)
    "G084",  # Nos últimos doze meses ___ recebe ou recebeu, algum cuidado em reabilitação de forma regular? (Por reabilitação quero dizer fisioterapia, terapia ocupacional, fonoaudiologia psicoterapia etc.)
]

modulo_J =  [
    # Utilização de Serviços de saude
    'J001', # estado de saude
    'J007', # doença cronica
    'J012', # Quantidade de consultas médicas no ultimos 12 meses
    "J00402", #  Qual foi o principal motivo de saúde que impediu ___ de realizar suas atividades habituais nas duas últimas semanas
]

modulo_L = [
    # Módulo L - Crianças com menos de dois anos
    "L019",  # Alguma vez ___________ recebeu Sulfato Ferroso? 
    "L021",  # Foi realizado o teste do pezinho?
    "L033",  # O teste do coraçãozinho realizado deu resultado alterado?
    "L037",  # Vacina Penta (também chamada de Pentavalente, DTP/Hib/HB)
    "L038",  # Vacina Poliomielite (também chamada de gotinha, VIP, VOP, PÓLIO, ANTIPÓLIO, POLIOMIELITE, SABIN)
    "L039",  # Vacina Pneumocócica (também chamada de Pneumo 10, Pneumo 13, Pncc)
    "L040",  # Vacina Tríplice Viral (também chamada SCR, TRIVIRAL, TV, MMR).
    "L01701",  # Leite materno?
    "L018",  # Desde que ___________nasceu, tomou ou comeu outro alimento que não leite materno?
    "L019",  # Alguma vez ___________ recebeu Sulfato Ferroso?
]
modulo_P = [
    # Módulo P - Estilos de Vida
    "P00103",  # Peso - Informado (em kg) (3 inteiros e 1 casa decimal)
    "P00104",  # Peso - Final (em kg) (3 inteiros e 1 casa decimal)
    "P00403",  # Altura - Informada (em cm) (3 inteiros)
    "P00404",  # Altura - Final (em cm) (3 inteiros)
    "P027",    # Com que frequência o(a) Sr(a) costuma consumir alguma bebida alcoólica?
    "P035",    # Quantos dias por semana o(a) Sr(a) costuma  (costumava)praticar exercício físico ou esporte?
    "P050", # Atualmente, o(a) Sr(a) fuma algum produto do tabaco?
    "P052", # E no passado, o(a) Sr(a) fumou algum produto do tabaco?
]
modulo_Q = [
    # Módulo Q - Doenças crônicas
    "Q00201",  # Algum médico já lhe deu o diagnóstico de hipertensão arterial (pressão alta)?
    "Q00202",  # Essa hipertensão arterial (pressão alta) ocorreu apenas durante algum período de gravidez?
    "Q003",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de hipertensão arterial (pressão alta)?
    "Q028",  # Em geral, em que grau a hipertensão ou alguma complicação da hipertensão limita as suas atividades habituais (como trabalhar, estudar, realizar afazeres domésticos etc.)?
    "Q03001",  # Algum médico já lhe deu o diagnóstico de diabetes?
    "Q03002",  # Esse diabetes ocorreu apenas durante algum período de gravidez?
    "Q031",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico do diabetes?
    "Q055011",  # Problemas na vista
    "Q055012",  # Infarto ou AVC (Acidente Vascular cerebral) /derrame ou outro problema circulatório
    "Q055013",  # Problema nos rins
    "Q060",  # Algum médico já lhe deu o diagnóstico de colesterol alto?
    "Q061",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de colesterol alto?
    "Q06306", # um médico já lhe deu o diagnóstico de uma doença do coração, tal como infarto, angina, insuficiência cardíaca ou outra?
    "Q06307",  # Infarto
    "Q06308",  # Angina
    "Q06309",  # Insuficiência cardíaca
    "Q06310",  # Arritmia
    "Q064",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico da doença do coração?
    "Q068",  # Algum médico já lhe deu o diagnóstico de AVC (Acidente Vascular Cerebral) ou derrame?
    "Q070",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico do derrame (ou AVC)?
    "Q074",  # Algum médico já lhe deu o diagnóstico de asma (ou bronquite asmática)?
    "Q075",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de asma?
    "Q079",  # Algum médico já lhe deu o diagnóstico de artrite ou reumatismo?
    "Q080",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de artrite ou reumatismo?
    "Q084",  # O(a) Sr(a) tem algum problema crônico de coluna, como dor crônica nas costas ou no pescoço, lombalgia, dor ciática, problemas nas vértebras ou disco?
    "Q085",  # Que idade o(a) Sr(a) tinha quando começou o problema na coluna?
    "Q087",  # Em geral, em que grau o problema na coluna limita as suas atividades habituais (tais como trabalhar, realizar afazeres domésticos, etc.)?
    "Q088",  # Algum médico já lhe deu o diagnóstico de DORT?
    "Q08901",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de DORT?
    "Q091",  # Em geral, em que grau o DORT limita as suas atividades habituais (tais como trabalhar, realizar afazeres domésticos etc.)?
    "Q092",  # Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?
    "Q09301",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de depressão?
    "Q109",  # Em geral, em que grau a depressão limita as suas atividades habituais (tais como trabalhar, realizar afazeres domésticos, etc.)?
    "Q11006", #Algum médico ou profissional de saúde (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de outra doença mental, como transtorno de ansiedade, síndrome do pânico, esquizofrenia, transtorno bipolar, psicose ou TOC (Transtorno Obsessivo Compulsivo) etc?
    "Q11007",  # Diagnóstico de Esquizofrenia
    "Q11008",  # Diagnóstico de Transtorno bipolar
    "Q11009",  # Diagnóstico de TOC (Transtorno obsessivo compulsivo)
    "Q111",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de doença mental?
    "Q11604",  # Algum médico já lhe deu o diagnóstico de alguma outra doença crônica no pulmão, tais como enfisema pulmonar, bronquite crônica ou DPOC (Doença Pulmonar Obstrutiva Crônica)?
    "Q11605",  # Diagnóstico de enfisema pulmonar
    "Q11606",  # Diagnóstico de bronquite crônica
    "Q11701",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico dessa(s) doença(s) no pulmão?
    "Q120",  # Algum médico já lhe deu diagnóstico de câncer?
    "Q12201",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de doença mental?
    "Q124",  # Algum médico já lhe deu o diagnóstico de insuficiência renal crônica?
    "Q125",  # Que idade o(a) Sr(a) tinha no primeiro diagnóstico de insuficiência renal crônica?
    "Q128", # Algum médico já lhe deu algum diagnóstico de outra doença crônica (física ou mental) ou doença de longa duração (de mais de 6 meses de duração)?
]
modulo_R = [
    # Módculo R - Saúde da Mulher - 15 anos ou mais
    "R011", # Segundo o médico, qual o motivo da retirada do útero?
    "R025",  # Com que idade a sra ficou menstruada pela primeira vez?
    "R026",  # A sra ainda fica menstruada?
    "R027",  # Com que idade a sra parou de menstruar?
    "R028",  # A sra já entrou na menopausa?
    "R029",  # Alguma vez a sra fez ou faz tratamento hormonal para alívio dos sintomas da menopausa (com comprimidos, adesivos, gel ou injeções)?
    "R034",  # A sra usa algum método para evitar a gravidez atualmente?
    "R035",  # Qual o principal motivo de não evitar a gravidez?
    "R03601",  # Pilula?
    "R037",  # A sra e/ou seu companheiro já fizeram ou fazem algum tratamento para engravidar?
    "R038",  # Há quanto tempo a sra está tentando engravidar?
]
modulo_S = [
    # Módulo S - Atendimento pré-natal e parto - 15 anos ou mais
    "S065",  # Alguma vez ficou grávida, mesmo que a gravidez não tenha chegado até o final?
    "S066",  # Quantos partos a Sra já teve?
    "S06703",  # Em que data foi o último parto? ANO
    "S068",  # Quando estava grávida fez alguma consulta de pré-natal?
    "S06901",  # Quantas tempo de gravidez tinha quando fez a primeira consulta pré-natal?
    "S06902",  # Quantas tempo de gravidez tinha quando fez a primeira consulta pré-natal?
    "S070",  # Quantas consultas de pré-natal fez durante esta gravidez?
    "S080",  # Durante o pré-natal de ______, foi realizado teste/exame para sífilis?
    "S082",  # Qual foi o resultado do teste/exame para sífilis?
    "S083",  # Recebeu tratamento para sífilis?
    "S086",  # Qual foi o resultado do teste/exame para sífilis do seu parceiro?
    "S087",  # O seu parceiro foi tratado?
    "S095",  # Durante o pré-natal a Sra foi orientada a usar preservativo?
    "S096",  # Quanto tempo antes do parto a Sra foi à última consulta do pré-natal?
    "S098",  # Quando estava grávida tomou alguma injeção para prevenir o bebê contra difteria e tétano (mal dos sete dias) – dT ou dTpa?
    "S099",  # Quantas doses dessa injeção tomou durante esta gravidez?
    "S09901",  # Número de doses
    "S110",  # Qual foi o peso dele(a) ao nascer?
    "S11001",  # Peso ao nascer (em quilogramas)
    "S112",  # Onde foi realizado o parto?
    "S115",  # Qual foi o tipo de parto?
    "S116",  # Nesta gravidez, entrou em trabalho de parto?
    "S117",  # Qual o principal motivo de ter tido parto cesáreo?
    "S118",  # Quantas semanas de gravidez tinha no momento do parto?
    "S11801",  # Quantidade de semanas
    "S125",  # Teve alguma complicação durante o parto?
    "S126",  # Você teve alguma complicação após o parto?
    "S127",  # Por causa dessa complicação precisou ser internada?
    "S130",  # Durante a primeira hora após o nascimento, seu(sua) filho(a) foi colocado(a) em contato com você pele a pele?
    "S131",  # Depois do nascimento, ele(a) ficou com você no quarto até a alta hospitalar?
    "S132",  # Seu(sua) filho(a) saiu do hospital junto com você?
    "S133",  # A criança não saiu do hospital junto com você porque:
    "S134",  # Você fez consulta de puerpério (consulta com médico ou enfermeiro até 42 dias após o parto)?
    "S136",  # Durante os primeiros três meses após o parto, você tomou medicamento contendo somente ferro ou ferro com vitaminas?
]

modulo_T = [
    # Módulo de DST
    "T004", #Nos últimos 12 meses, algum médico lhe deu diagnóstico de doença/infecção sexualmente transmissível?

]
modulo_W = [
    # Módulo W - Antropometria
    "W001",  # Antropometria aferida do morador selecionado
    "W00101",  # Peso - 1ª pesagem (em kg)(3 inteiros e 1 casa decimal)
    "W00102",  # Peso - 2ª pesagem (em kg)(3 inteiros e 1 casa decimal)
    "W00103",  # Peso - Final (em kg)(3 inteiros e 1 casa decimal)
    "W00201",  # Altura - 1ª medição (em cm)(3 inteiros e 1 casa decimal)
    "W00202",  # Altura - 2ª medição (em cm)(3 inteiros e 1 casa decimal)
    "W00203",  # Altura - Final (em cm)(3 inteiros e 1 casa decimal)
    "VDL001",  # Idade em meses das crianças nascidas no período de referência do Módulo L
]

modulo_derivadas = [
    "VDE014", #Grupamentos de atividade do trabalho principal da semana de referência para pessoas de 14 anos ou mais de idade
    "VDF004", # Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do
]

In [3]:
colunas_interesse = modulo_identificacao + modulo_C + modulo_D + modulo_E + modulo_G + modulo_J + modulo_L + modulo_P + modulo_Q + modulo_R + modulo_S + modulo_T + modulo_W + modulo_derivadas

In [4]:

dataframes = []

for i in tqdm(range(21)):  # Arquivos de 0 a 20
    df = pd.read_csv(f"https://raw.githubusercontent.com/LeandroCoelhos/Notebook_PNS/refs/heads/main/data/pns2019_{i}.csv", usecols=colunas_interesse)
    #df = pd.read_csv(f"pns2019_{i}.csv")
    dataframes.append(df)
# Armazenar em um unico Dataframe
pns_2019 = pd.concat(dataframes, ignore_index=True)
pns_2019.info(show_counts=True, verbose=True)

100%|██████████| 21/21 [00:41<00:00,  1.95s/it]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293726 entries, 0 to 293725
Data columns (total 169 columns):
 #    Column     Non-Null Count   Dtype  
---   ------     --------------   -----  
 0    V0001      293726 non-null  int64  
 1    V0024      293726 non-null  int64  
 2    UPA_PNS    293726 non-null  int64  
 3    V0006_PNS  293726 non-null  int64  
 4    V0015      293726 non-null  int64  
 5    V0022      279382 non-null  float64
 6    V0026      293726 non-null  int64  
 7    V0031      293726 non-null  int64  
 8    V0025A     279382 non-null  float64
 9    V0025B     20712 non-null   float64
 10   C001       279382 non-null  float64
 11   C00301     279382 non-null  float64
 12   C006       279382 non-null  float64
 13   C00701     279382 non-null  float64
 14   C00702     279382 non-null  float64
 15   C00703     279382 non-null  float64
 16   C008       279382 non-null  float64
 17   C009       279382 non-null  float64
 18   D00201     279382 non-null  float64
 19   

In [5]:
pns_2019['chave_domicio'] = pns_2019['V0001'].astype(str) + '_' + pns_2019['V0024'].astype(str) + '_' + pns_2019['UPA_PNS'].astype(str) + '_' + pns_2019['V0006_PNS'].astype(str)
pns_2019['chave_pessoa']  = pns_2019['V0001'].astype(str) + '_' + pns_2019['V0024'].astype(str) + '_' + pns_2019['UPA_PNS'].astype(str) + '_' + pns_2019['V0006_PNS'].astype(str) + '_' + pns_2019['C00301'].astype(str)
pns_2019['peso_morador_selec'] = pns_2019['V00291']*(94114 / 168426190)
pns_2019['peso_morador_descalibrado'] = pns_2019['V0029']

In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13726 entries, 0 to 13725
Data columns (total 169 columns):
 #    Column     Non-Null Count  Dtype  
---   ------     --------------  -----  
 0    V0001      13726 non-null  int64  
 1    V0024      13726 non-null  int64  
 2    UPA_PNS    13726 non-null  int64  
 3    V0006_PNS  13726 non-null  int64  
 4    V0015      13726 non-null  int64  
 5    V0022      12980 non-null  float64
 6    V0026      13726 non-null  int64  
 7    V0031      13726 non-null  int64  
 8    V0025A     12980 non-null  float64
 9    V0025B     1071 non-null   float64
 10   C001       12980 non-null  float64
 11   C00301     12980 non-null  float64
 12   C006       12980 non-null  float64
 13   C00701     12980 non-null  float64
 14   C00702     12980 non-null  float64
 15   C00703     12980 non-null  float64
 16   C008       12980 non-null  float64
 17   C009       12980 non-null  float64
 18   D00201     12980 non-null  float64
 19   D00901     8460 non-nul

In [7]:
df = pns_2019.copy()
# Dicionário para renomear as colunas
rename_columns = {
    "V0001": "UF",
    "V0024":"Estrato",
    "UPA_PNS":"UPA",
    "V0006_PNS":"N_ordem_domicilio_PNS",
    "C00301":"N_ordem_morador",
    "C001":"N_pessoas_domicilio",
    "V0022":"Quantidade_moradores",
    "V0015":'tipo_entrevista',
    "V0026": "regiao_metropolitana",
    "V0031": "capital_metropolitana",
    "C006": "sexo",
    "C00701": "dia_nascimento",
    "C00702": "mes_nascimento",
    "C00703": "ano_nascimento",
    "C008": "idade",
    "P00103": "peso_info_kg",
    "P00104": "peso_final_kg",
    "P00403": "altura_info_cm",
    "P00404": "altura_final_cm",
    "W001": "antropometria_aferida_morador",
    "W00101": "peso_1_pesagem_kg",
    "W00102": "peso_2_pesagem_kg",
    "W00103": "peso_final_medido_kg",
    "W00201": "altura_1_medicao_cm",
    "W00202": "altura_2_medicao_cm",
    "W00203": "altura_final_medida_cm",
}

# Renomear as colunas no DataFrame
# df.rename(columns=rename_columns, inplace=True)

# Iterar sobre as colunas do dicionário e criar novas colunas
for original_col, new_col in rename_columns.items():
    if original_col in df.columns:  # Verificar se a coluna original existe no DataFrame
        df[new_col] = df[original_col]  # Criar a nova coluna com os mesmos valores da original


In [8]:
# Preencher a coluna de estado
estados = {
    11: "Rondônia",
    12: "Acre",
    13: "Amazonas",
    14: "Roraima",
    15: "Pará",
    16: "Amapá",
    17: "Tocantins",
    21: "Maranhão",
    22: "Piauí",
    23: "Ceará",
    24: "Rio Grande do Norte",
    25: "Paraíba",
    26: "Pernambuco",
    27: "Alagoas",
    28: "Sergipe",
    29: "Bahia",
    31: "Minas Gerais",
    32: "Espírito Santo",
    33: "Rio de Janeiro",
    35: "São Paulo",
    41: "Paraná",
    42: "Santa Catarina",
    43: "Rio Grande do Sul",
    50: "Mato Grosso do Sul",
    51: "Mato Grosso",
    52: "Goiás",
    53: "Distrito Federal",
}

df['UF_sigla'] = df['UF'].map(estados)

# Mapeamento direto de estados para suas macroregiões
estado_para_regiao = {
    "Rondônia": "Norte",
    "Acre": "Norte",
    "Amazonas": "Norte",
    "Roraima": "Norte",
    "Pará": "Norte",
    "Amapá": "Norte",
    "Tocantins": "Norte",
    "Maranhão": "Nordeste",
    "Piauí": "Nordeste",
    "Ceará": "Nordeste",
    "Rio Grande do Norte": "Nordeste",
    "Paraíba": "Nordeste",
    "Pernambuco": "Nordeste",
    "Alagoas": "Nordeste",
    "Sergipe": "Nordeste",
    "Bahia": "Nordeste",
    "Minas Gerais": "Sudeste",
    "Espírito Santo": "Sudeste",
    "Rio de Janeiro": "Sudeste",
    "São Paulo": "Sudeste",
    "Paraná": "Sul",
    "Santa Catarina": "Sul",
    "Rio Grande do Sul": "Sul",
    "Mato Grosso do Sul": "Centro-Oeste",
    "Mato Grosso": "Centro-Oeste",
    "Goiás": "Centro-Oeste",
    "Distrito Federal": "Centro-Oeste",
}

# Adicionar a coluna 'regiao_brasileira' ao DataFrame
df['regiao_brasileira'] = df['UF_sigla'].map(estado_para_regiao)

#Mapear região
regiao = {
    1: "Urbano",
    2: "Rural",
}
df['regiao_metropolitana'] = df['regiao_metropolitana'].map(regiao)

# Dicionário de mapeamento
regioes_map = {
    1: "Capital",
    2: "Região Metropolitana, excluindo a capital",
    3: "RIDE, excluindo a capital",
    4: "Interior"
}
df['capital_metropolitana'] = df['capital_metropolitana'].map(regioes_map)

# Dicionário de Sexo
sexo = {
    1: "Masculino",
    2: "Feminino"
}
df['sexo'] = df['sexo'].map(sexo)

# Pesquisa realizada
status_domicilios = {
    1: "Realizada",
    2: "Domicílio Fechado",
    3: "Recusa",
    4: "Outro Motivo",
    5: "Domicílio Vago",
    6: "Domicílio de uso ocasional",
    7: "Domicílio coletivo ou improvisado",
    8: "Domicílio em obras ou em ruínas",
    9: "Domicílio demolido",
    10: "Domicílio não encontrado",
    11: "Unidade não residencial",
    12: "Domicílio fora do setor",
}
df['tipo_entrevista'] = df['tipo_entrevista'].map(status_domicilios)

In [9]:
df.shape, df.drop_duplicates().shape

((293726, 201), (293726, 201))

In [10]:
df[df['idade'] < 15]['peso_morador_selec'].value_counts(dropna=False)

peso_morador_selec
NaN    58287
Name: count, dtype: int64

In [11]:
cols = df.columns
for col in cols:
  print(df[col].value_counts(dropna=False))
  print('\n\n')

V0001
35    18504
21    17912
31    15421
23    15281
33    14776
15    14393
13    13074
26    12381
41    11694
29    11061
32    10673
42    10607
27    10474
43    10369
25    10094
24     9964
22     9044
50     8766
52     8506
14     8476
12     8270
28     8140
53     7609
51     7607
11     7391
16     6640
17     6599
Name: count, dtype: int64



V0024
1210010    3691
1410011    3138
2710111    2720
2410011    2569
1250020    2484
           ... 
3220020      61
4230020      54
3321020      54
4220020      47
3532020      45
Name: count, Length: 574, dtype: int64



UPA_PNS
130033536    94
160007292    93
140004603    88
140008399    87
140002449    86
             ..
350286470    14
410182027    14
270018200    14
210092069    13
230030392    12
Name: count, Length: 8031, dtype: int64



V0006_PNS
2     21905
7     21764
11    21746
1     21743
12    21723
10    21707
3     21643
6     21587
9     21552
5     21525
8     21503
4     21208
13    10242
15    10228
14    10099


# Dados não informados na pesquisa

5% - 16342 não informaram Data de nascimento

67,8% sem informação de peso e Altura

Sessão W - 2,5% dos entrevistados foram Aferidos

# Limpeza

- Somente entrevistas realizadas
- Informações de Idade


In [12]:
print('Antes:', df.shape)
# Entrevistas realizadas
df = df[df['tipo_entrevista']=='Realizada'].reset_index(drop=True)
print('Depois:', df.shape)

Antes: (293726, 201)
Depois: (279382, 201)


In [13]:
print('Antes:', df.shape)
# Idade Informada
df = df[~df['idade'].isna()].reset_index(drop=True)
print('Depois:', df.shape)

Antes: (279382, 201)
Depois: (279382, 201)


# Criação de novas Features

## Ano Nascimento Inferido pela idade

In [14]:
df['ano_nascimento_inferido'] = np.where(df['ano_nascimento']==9999, 2019 - df['idade'], df['ano_nascimento'])

## IMC

In [15]:
# Calcular o IMC
# Função para calcular o IMC
def calcular_imc(row):
    """
    Calcula o IMC com base no peso (kg) e altura (cm).
    :param row: Linha do DataFrame.
    :return: Valor do IMC (float) ou None se peso ou altura forem nulos.
    """
    if pd.notnull(row['peso']) and pd.notnull(row['altura']) and row['altura'] > 0:
        altura_metros = row['altura'] / 100  # Converter altura de cm para metros
        return row['peso'] / (altura_metros ** 2)
    return None

# Função para classificar o IMC
def classificar_imc(imc):
    if pd.isnull(imc):
        return None, None
    elif imc < 18.5:
        return "Magreza", 0
    elif 18.5 <= imc < 25:
        return "Normal", 0
    elif 25.0 <= imc < 30:
        return "Sobrepeso", 1
    elif 30.0 <= imc < 40:
        return "Obesidade", 2
    elif imc >= 40.0:
        return "Obesidade Grave", 3
    return None, None


In [16]:
# Peso unico, usar medido, ou informado
df['peso'] = np.where(df['peso_final_kg'].isna(), df['peso_info_kg'], df['peso_final_kg'])
# Peso Altura, usar medido, ou informado
df['altura'] = np.where(df['altura_final_cm'].isna(), df['altura_info_cm'], df['altura_final_cm'])

# Aplicar a função ao DataFrame
df['imc'] = df.apply(calcular_imc, axis=1)

# Aplicar a função e criar as colunas 'imc_classificacao' e 'imc_grau_obesidade'
df[['imc_classificacao', 'imc_grau_obesidade']] = df['imc'].apply(
    lambda x: pd.Series(classificar_imc(x))
)

## Deficiencias

In [17]:
modulo =[
    'G033',
    # 'G047',
 'G049',
 'G051',
 'G053',
 'G059',
 'G060',
 'G062',
 'G064',
 'G066',
#  'G070',
 'G072',
 'G073',
#  'G079',
#  'G083',
 'G084',
 'modulo_pre'
 ]

a = {
    "G047":{
        1: "Não, nenhuma dificuldade",
        2: "Sim, alguma dificuldade",
        3: "Sim, muita dificuldade",
        4: "Sim, não consegue de modo algum",
        9: "Ignorado"
    },
    "G070":{
        1: "Não, nenhuma dificuldade",
        2: "Sim, alguma dificuldade",
        3: "Sim, muita dificuldade",
        4: "Sim, não consegue de modo algum",
        9: "Ignorado"
    },
    "G079":{
        1: "Não, nenhuma dificuldade",
        2: "Sim, alguma dificuldade",
        3: "Sim, muita dificuldade",
        4: "Sim, não consegue de modo algum",
        9: "Ignorado"
    },
    "G083":{
        1: "Não, nenhuma dificuldade",
        2: "Sim, alguma dificuldade",
        3: "Sim, muita dificuldade",
        4: "Sim, não consegue de modo algum",
        9: "Ignorado"
    },

}

modulo_pre =[
    'G047',
    'G070',
    'G079',
    'G083',
 ]

df['modulo_pre'] = np.where(
    df[modulo_pre].isin([2,3,4]).any(axis=1),
    1,  # tem
    0   # não tem
)

In [18]:
df['deficiencia'] = np.where(
    df[modulo].isin([1]).any(axis=1),
    1,  # tem
    0   # não tem
)
df['deficiencia'].value_counts(dropna=False)

deficiencia
0    147135
1    132247
Name: count, dtype: int64

## Vacinas

In [19]:
modulo_L = [
    # Módulo L - Crianças com menos de dois anos
    "L021",  # Foi realizado o teste do pezinho?
    "L033",  # O teste do coraçãozinho realizado deu resultado alterado?
    "L037",  # Vacina Penta (também chamada de Pentavalente, DTP/Hib/HB)
    "L038",  # Vacina Poliomielite (também chamada de gotinha, VIP, VOP, PÓLIO, ANTIPÓLIO, POLIOMIELITE, SABIN)
    "L039",  # Vacina Pneumocócica (também chamada de Pneumo 10, Pneumo 13, Pncc)
    "L040",  # Vacina Tríplice Viral (também chamada SCR, TRIVIRAL, TV, MMR).
    "L01701",  # Leite materno?
    "L018",  # Desde que ___________nasceu, tomou ou comeu outro alimento que não leite materno?
    "L019",  # Alguma vez ___________ recebeu Sulfato Ferroso?
]

In [20]:
vac_cols = ['L037','L038','L039','L040']
for col in vac_cols:
    df[col + '_bin'] = (df[col] == 1).astype(int)  # Se a codificação 1=Sim, 2=Não, etc.

# Exemplo: número de vacinas:
df['num_vacinas'] = df[[c + '_bin' for c in vac_cols]].sum(axis=1)

# Cobertura "completa" (tomou as 4 vacinas):
df['cobertura_completa'] = (df['num_vacinas'] == 4).astype(int)

In [21]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279382 entries, 0 to 279381
Data columns (total 215 columns):
 #    Column                         Non-Null Count   Dtype  
---   ------                         --------------   -----  
 0    V0001                          279382 non-null  int64  
 1    V0024                          279382 non-null  int64  
 2    UPA_PNS                        279382 non-null  int64  
 3    V0006_PNS                      279382 non-null  int64  
 4    V0015                          279382 non-null  int64  
 5    V0022                          279382 non-null  float64
 6    V0026                          279382 non-null  int64  
 7    V0031                          279382 non-null  int64  
 8    V0025A                         279382 non-null  float64
 9    V0025B                         20712 non-null   float64
 10   C001                           279382 non-null  float64
 11   C00301                         279382 non-null  float64
 12   C006          

# Vacinas - new

Não rodei ainda, precisa testar

In [22]:
# Idades e vacinas:

# Checa se calendário vacinal é adequado até os 6 anos e cria uma coluna
# Somente os anos completos foram considerados na idade, sem os meses e dias
# A conta foi inclusiva:
# as vacinas de 2 meses, se não aplicadas em uma criança de 11 meses, ainda são consideradas do 1º ano
# e, portanto, o esquema vacinal está adequado

# idade: numero mínimo de status vacinal para esta idade, conforme dicionário
# Considerei registro ilegível como válido

N_minimo_doses = {
    # penta
    # D1 aos 2 meses, D2 aos 4, D3 aos 6 meses; reforço com 15m e depois com 4a.
    'L037': {
        0: 1, # minimo 1 dose
        1: 4, # minimo 3 doses
        2: 5, # minimo 1o reforço
        3: 5,
        4: 5,
        5: 6, #minimo 2o reforço
        6: 6,
    },

    # Vacina da polio (VIP/VOP)
    # D1 aos 2 meses, d2 4m, d3 6m, r1 15m, r2 4a
    'L038' : {
        0: 1, # minimo 1 dose
        1: 4, # minimo 3 doses
        2: 5, # minimo 1o reforço
        3: 5,
        4: 5,
        5: 6, #minimo 2o reforço
        6: 6,
    },

    # pneumo
    # d1 2m, d2 4m, reforço 12m
    'L039' : {
        0: 1,
        1: 4, # 1o reforço aos 12m
        2: 4,
        3: 4,
        4: 4,
        5: 4,
        6: 4,
    },

    # Triplice viral
    # D1 aos 12 meses, reforço na adolescência
    'L040' : {
        0: 1,
        1: 2, # D1 aos 12 meses
        2: 2,
        3: 2,
        4: 2,
        5: 2,
        6: 2,
    }
}

def checar_estado_vacinal(idade, coluna_vacinas, N_minimo):
  if idade not in N_minimo:
    return np.nan

  # pega o estado vacinal da coluna
  estado_vacinal = coluna_vacinas

  # Compara se o número de doses é maior que o limiar para a idade
  return estado_vacinal >= N_minimo[idade]

# Aplica a lógica para cada coluna de vacina e cria nova coluna com estado válido ou não
for coluna_vacina in ['L037', 'L038', 'L039', 'L040']:
    df[f'{coluna_vacina}_status'] = df.apply(
        lambda row: checar_estado_vacinal(row['idade'], row[coluna_vacina], N_minimo_doses[coluna_vacina]), axis=1
    )

# Cria uma coluna final para verificar se todas as vacinas estão adequadas
df['estado_vacinal'] = df[['L037_status', 'L038_status', 'L039_status', 'L040_status']].fillna(False).all(axis=1)

# Exibe o resumo do estado vacinal
print(df['estado_vacinal'].value_counts(dropna=False, sort=False))

estado_vacinal
False    276030
True       3352
Name: count, dtype: int64


C:\Users\leand\AppData\Local\Temp\ipykernel_13700\1447677503.py:79: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['estado_vacinal'] = df[['L037_status', 'L038_status', 'L039_status', 'L040_status']].fillna(False).all(axis=1)


# Clusterização

## Grupo de Crianças de Até 6 anos de IDADE

variavel de Idade para segmentar - Referencia temporal das respostas na data da entrevista

### Verificação das idades

In [23]:
pd.set_option('display.max_rows', 100)

In [24]:
# Seguir com a variavel de Idade para segmentar - Referencia temporal das respostas na data da entrevista
df[(df.ano_nascimento>= 2012)&(df.ano_nascimento< 9999)]['idade'].value_counts(dropna=False)

idade
6.0    3616
4.0    3582
5.0    3508
3.0    3464
1.0    3411
0.0    3349
2.0    3270
7.0    2150
Name: count, dtype: int64

In [25]:
df[df.idade <= 6]['idade'].value_counts(dropna=False)

idade
6.0    3838
4.0    3807
5.0    3728
3.0    3665
1.0    3534
2.0    3432
0.0    3349
Name: count, dtype: int64

In [26]:
df[df.idade <= 6]['ano_nascimento'].value_counts(dropna=False)

ano_nascimento
2014.0    3618
2015.0    3558
2017.0    3474
2013.0    3463
2018.0    3272
2016.0    3265
2019.0    1962
2012.0    1588
9999.0    1153
Name: count, dtype: int64

### Filtro de crianças

In [27]:
df.columns

Index(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'V0015', 'V0022', 'V0026',
       'V0031', 'V0025A', 'V0025B',
       ...
       'L038_bin', 'L039_bin', 'L040_bin', 'num_vacinas', 'cobertura_completa',
       'L037_status', 'L038_status', 'L039_status', 'L040_status',
       'estado_vacinal'],
      dtype='object', length=220)

In [28]:
colunas_criadas = ['Quantidade_moradores', 'chave_domicio', 'chave_pessoa', 'UF_sigla', 'regiao_brasileira', 'regiao_metropolitana',  'sexo',
                   'dia_nascimento', 'mes_nascimento', 'ano_nascimento', 'idade', 'peso', 'altura', 'peso_morador_selec', 'peso_morador_descalibrado','deficiencia', 'modulo_pre',
        'num_vacinas', 'cobertura_completa']

colunas_interesse_children = modulo_identificacao + colunas_criadas + modulo_C + modulo_D + modulo_G + modulo_L + modulo_P

In [29]:
df_children = df[(df.idade <= 6)].copy()

# df_children = df_children[colunas_interesse_children]

In [30]:
df_children.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25353 entries, 4 to 279373
Data columns (total 220 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    V0001                          25353 non-null  int64  
 1    V0024                          25353 non-null  int64  
 2    UPA_PNS                        25353 non-null  int64  
 3    V0006_PNS                      25353 non-null  int64  
 4    V0015                          25353 non-null  int64  
 5    V0022                          25353 non-null  float64
 6    V0026                          25353 non-null  int64  
 7    V0031                          25353 non-null  int64  
 8    V0025A                         25353 non-null  float64
 9    V0025B                         1871 non-null   float64
 10   C001                           25353 non-null  float64
 11   C00301                         25353 non-null  float64
 12   C006                           253

In [31]:
dicionario_1 = {
    "V0001": "Unidade da Federação",
    "V0026": "Tipo de situação censitária",
    "C001": "Quantidade de pessoas no domicílio",
    "C006": "Sexo",
    "C00701": "Dia de nascimento",
    "C00702": "Mês de nascimento",
    "C00703": "Ano de nascimento",
    "C008": "Idade do morador na data de referência",
    "D00201": "Frequenta escola ou creche",
    "D00901": "Qual foi o curso mais elevado que ___frequentou",
    "G046": "___ tem dificuldade permanente de enxergar mesmo usando óculos, lentes de contato ou lupas?",
    "G049": "____________ faz uso de aparelho auditivo",
    "G051": "____________ faz uso de implante coclear?",
    "G053": "____________ faz uso de sistema de frequência modulada individual (sistema FM)?",
    "G059": "___ usa algum aparelho de auxílio para se locomover?",
    "G060": "____________ faz uso de cadeira de rodas?",
    "G062": "____________ faz uso de bengala, muletas ou andador?",
    "G064": "____________ faz uso de prótese?",
    "G066": "____________ faz uso de órtese?",
    "G070": "___ tem dificuldade permanente de caminhar ou subir degraus?",
    "G072": "___ usa algum aparelho de auxílio para realizar movimentos com os membros superiores?",
    "G073": "____________ faz uso de prótese para os membros superiores?",
    "G079": "___tem dificuldade permanente para levantar uma garrafa com dois litros de água da cintura até a altura dos olhos?",
    "G083": "Por causa de alguma limitação nas funções mentais ou intelectuais, _________ tem dificuldade permanente para realizar atividades habituais.",
    "G084": "Nos últimos doze meses ___ recebe ou recebeu, algum cuidado em reabilitação de forma regular?",
    "L021": "Foi realizado o teste do pezinho?",
    "L033": "O teste do coraçãozinho realizado deu resultado alterado?",
    "L037": "Vacina Penta (também chamada de Pentavalente, DTP/Hib/HB)",
    "L038": "Vacina Poliomielite (também chamada de gotinha, VIP, VOP, PÓLIO, ANTIPÓLIO, POLIOMIELITE, SABIN)",
    "L039": "Vacina Pneumocócica (também chamada de Pneumo 10, Pneumo 13, Pncc)",
    "L040": "Vacina Tríplice Viral (também chamada SCR, TRIVIRAL, TV, MMR).",
    "L01701": "Leite materno?",
    "L018": "Desde que ___________nasceu, tomou ou comeu outro alimento que não leite materno?",
    "L019": "Alguma vez ___________ recebeu Sulfato Ferroso?",
    "P00103": "Peso - Informado (em kg) (3 inteiros e 1 casa decimal)",
    "P00104": "Peso - Final (em kg) (3 inteiros e 1 casa decimal)",
    "P00403": "Altura - Informada (em cm) (3 inteiros)",
    "P00404": "Altura - Final (em cm) (3 inteiros)",
    "Q00201": "Algum médico já lhe deu o diagnóstico de hipertensão arterial (pressão alta)?",
    "Q00202": "Essa hipertensão arterial (pressão alta) ocorreu apenas durante algum período de gravidez?",
    "Q003": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de hipertensão arterial (pressão alta)?",
    "Q028": "Em geral, em que grau a hipertensão limita as suas atividades habituais?",
    "Q03001": "Algum médico já lhe deu o diagnóstico de diabetes?",
    "Q03002": "Esse diabetes ocorreu apenas durante algum período de gravidez?",
    "Q031": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico do diabetes?",
}

dicionario_2 = {
    "Q055011": "Problemas na vista",
    "Q055012": "Infarto ou AVC (Acidente Vascular cerebral) / derrame ou outro problema circulatório",
    "Q055013": "Problema nos rins",
    "Q060": "Algum médico já lhe deu o diagnóstico de colesterol alto?",
    "Q061": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de colesterol alto?",
    "Q06307": "Infarto",
    "Q06308": "Angina",
    "Q06309": "Insuficiência cardíaca",
    "Q06310": "Arritmia",
    "Q064": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico da doença do coração?",
    "Q068": "Algum médico já lhe deu o diagnóstico de AVC (Acidente Vascular Cerebral) ou derrame?",
    "Q070": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico do derrame (ou AVC)?",
    "Q074": "Algum médico já lhe deu o diagnóstico de asma (ou bronquite asmática)?",
    "Q075": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de asma?",
    "Q079": "Algum médico já lhe deu o diagnóstico de artrite ou reumatismo?",
    "Q080": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de artrite ou reumatismo?",
    "Q084": "O(a) Sr(a) tem algum problema crônico de coluna, como dor crônica nas costas ou no pescoço?",
    "Q085": "Que idade o(a) Sr(a) tinha quando começou o problema na coluna?",
    "Q087": "Em geral, em que grau o problema na coluna limita as suas atividades habituais?",
    "Q088": "Algum médico já lhe deu o diagnóstico de DORT?",
    "Q08901": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de DORT?",
    "Q091": "Em geral, em que grau o DORT limita as suas atividades habituais?",
    "Q092": "Algum médico ou profissional de saúde mental já lhe deu o diagnóstico de depressão?",
    "Q09301": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de depressão?",
    "Q109": "Em geral, em que grau a depressão limita as suas atividades habituais?",
    "Q11007": "Diagnóstico de Esquizofrenia",
    "Q11008": "Diagnóstico de Transtorno bipolar",
    "Q11009": "Diagnóstico de TOC (Transtorno obsessivo compulsivo)",
    "Q111": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de doença mental?",
    "Q11604": "Algum médico já lhe deu o diagnóstico de alguma outra doença crônica no pulmão?",
    "Q11605": "Diagnóstico de enfisema pulmonar",
    "Q11606": "Diagnóstico de bronquite crônica",
    "Q11701": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico dessa(s) doença(s) no pulmão?",
    "Q120": "Algum médico já lhe deu diagnóstico de câncer?",
    "Q12201": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de doença mental?",
    "Q124": "Algum médico já lhe deu o diagnóstico de insuficiência renal crônica?",
    "Q125": "Que idade o(a) Sr(a) tinha no primeiro diagnóstico de insuficiência renal crônica?",
    "R025": "Com que idade a sra ficou menstruada pela primeira vez?",
    "R026": "A sra ainda fica menstruada?",
    "R027": "Com que idade a sra parou de menstruar?",
    "R028": "A sra já entrou na menopausa?",
    "R029": "Alguma vez a sra fez ou faz tratamento hormonal para alívio dos sintomas da menopausa?",
    "R034": "A sra usa algum método para evitar a gravidez atualmente?",
    "R035": "Qual o principal motivo de não evitar a gravidez?",
    "R03601": "Pilula?",
    "R037": "A sra e/ou seu companheiro já fizeram ou fazem algum tratamento para engravidar?",
    "R038": "Há quanto tempo a sra está tentando engravidar?",
    "S065": "Alguma vez ficou grávida, mesmo que a gravidez não tenha chegado até o final?",
    "S066": "Quantos partos a Sra já teve?",
    "S06701": "Em que data foi o último parto?",
    "S068": "Quando estava grávida fez alguma consulta de pré-natal?",
    "S070": "Quantas consultas de pré-natal fez durante esta gravidez?",
    "S082": "Qual foi o resultado do teste/exame para sífilis?",
    "S083": "Recebeu tratamento para sífilis?",
    "S086": "Qual foi o resultado do teste/exame para sífilis do seu parceiro?",
    "S087": "O seu parceiro foi tratado?",
    "S095": "Durante o pré-natal a Sra foi orientada a usar preservativo?",
    "S096": "Quanto tempo antes do parto a Sra foi à última consulta do pré-natal?",
    "S098": "Quando estava grávida tomou alguma injeção para prevenir o bebê contra difteria e tétano?",
    "S099": "Quantas doses dessa injeção tomou durante esta gravidez?",
    "S110": "Qual foi o peso dele(a) ao nascer?",
    "S112": "Onde foi realizado o parto?",
    "S115": "Qual foi o tipo de parto?",
    "S116": "Nesta gravidez, entrou em trabalho de parto?",
    "S117": "Qual o principal motivo de ter tido parto cesáreo?",
    "S125": "Teve alguma complicação durante o parto?",
    "S127": "Por causa dessa complicação precisou ser internada?",
    "W001": "Antropometria aferida do morador selecionado",
    "W00203": "Altura - Final (em cm)(3 inteiros e 1 casa decimal)",
    "VDL001": "Idade em meses das crianças nascidas no período de referência do Módulo L",
}

dicionarios = dicionario_1 | dicionario_2

In [32]:
for col in colunas_interesse_children:
  if col in dicionarios:  # Verificar se `col` está no dicionário
        print(f"Chave: {col}, Valor: {dicionarios[col]}")
  print(df_children[col].value_counts(dropna=False))
  print('\n\n')

Chave: V0001, Valor: Unidade da Federação
V0001
21    1858
13    1580
35    1412
15    1374
23    1248
31    1149
26    1049
33     993
14     990
41     929
27     910
29     892
32     866
12     831
25     815
50     812
24     794
22     724
16     721
43     712
42     709
11     706
28     705
51     693
52     692
53     613
17     576
Name: count, dtype: int64



V0024
1410011    347
1210010    319
1250020    288
1450020    269
1310212    223
          ... 
3321020      3
4230020      3
4253013      3
4355013      2
4220020      1
Name: count, Length: 574, dtype: int64



UPA_PNS
160007292    20
140002449    19
130048910    18
130033536    18
140001528    18
             ..
530032771     1
530014348     1
110018052     1
110017394     1
260001626     1
Name: count, Length: 7135, dtype: int64



V0006_PNS
7     1968
12    1897
3     1888
5     1860
1     1859
10    1848
11    1834
2     1828
6     1820
9     1809
8     1790
4     1700
15    1016
13     944
14     923
17     130


In [33]:
6311 + 280 + 41

6632

In [34]:
df_children

,V0001,V0024,UPA_PNS,V0006_PNS,V0015,V0022,V0026,V0031,V0025A,V0025B,...,L038_bin,L039_bin,L040_bin,num_vacinas,cobertura_completa,L037_status,L038_status,L039_status,L040_status,estado_vacinal
4,11,1110011,110000016,1,1,6.0,1,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
5,11,1110011,110000016,1,1,6.0,1,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
16,11,1110011,110000016,3,1,8.0,1,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
17,11,1110011,110000016,3,1,8.0,1,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
20,11,1110011,110000016,5,1,2.0,1,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279358,53,5310220,530044434,6,1,5.0,2,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
279363,53,5310220,530044434,8,1,6.0,2,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False
279364,53,5310220,530044434,8,1,6.0,2,1,9.0,NaN,...,0,0,0,0,0,True,True,True,True,True
279372,53,5310220,530044434,11,1,4.0,2,1,9.0,NaN,...,0,0,0,0,0,False,False,False,False,False


In [35]:
df_children.to_csv('data/df_criancas_v2.csv', index=False)

In [36]:
df_children['estado_vacinal'].value_counts(dropna=False, sort=False)

estado_vacinal
False    22001
True      3352
Name: count, dtype: int64

In [ ]:
df_children['peso_morador_selec'].value_counts(dropna=False)

peso_morador_selec
NaN    25353
Name: count, dtype: int64

In [ ]:
df_children.columns

Index(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'C00301', 'V0015', 'V0022',
       'V0026', 'V0031', 'V0025A', 'V0025B', 'V0029', 'V00291', 'V00282',
       'V00283', 'V00293', 'V00303', 'Quantidade_moradores', 'chave_domicio',
       'chave_pessoa', 'UF_sigla', 'regiao_brasileira', 'regiao_metropolitana',
       'sexo', 'dia_nascimento', 'mes_nascimento', 'ano_nascimento', 'idade',
       'peso', 'altura', 'peso_morador_selec', 'peso_morador_descalibrado',
       'deficiencia', 'modulo_pre', 'num_vacinas', 'cobertura_completa',
       'C001', 'C006', 'C00701', 'C00702', 'C00703', 'C008', 'D00201',
       'D00901', 'G046', 'G049', 'G051', 'G053', 'G059', 'G060', 'G062',
       'G064', 'G066', 'G070', 'G072', 'G073', 'G079', 'G083', 'G084', 'L021',
       'L033', 'L037', 'L038', 'L039', 'L040', 'L01701', 'L018', 'L019',
       'P00103', 'P00104', 'P00403', 'P00404'],
      dtype='object')

In [ ]:
df_children['V00283'].value_counts(dropna=False)

V00283
214.0    1501
154.0     886
134.0     781
234.0     712
531.0     613
351.0     603
264.0     593
131.0     577
141.0     565
314.0     560
294.0     521
424.0     518
354.0     509
124.0     473
274.0     454
504.0     445
254.0     445
244.0     436
414.0     434
144.0     425
332.0     410
174.0     388
224.0     375
114.0     373
501.0     367
322.0     367
121.0     358
331.0     356
271.0     356
161.0     355
514.0     353
434.0     353
284.0     349
111.0     333
352.0     300
311.0     298
324.0     294
151.0     289
232.0     275
412.0     275
251.0     268
231.0     261
221.0     258
521.0     255
211.0     249
312.0     234
261.0     233
291.0     228
334.0     227
281.0     225
524.0     224
262.0     223
132.0     222
241.0     222
411.0     220
511.0     215
321.0     205
431.0     203
152.0     199
164.0     199
171.0     188
162.0     167
432.0     156
292.0     143
242.0     136
421.0     135
282.0     131
522.0     127
512.0     125
252.0     102
272.0     100